In [ ]:
############ modified from https://github.com/jkrohn5/Code-Snips/blob/main/downloadfloodhazard.py #################

import os
import requests
import json
import re
import io
import zipfile
import arcpy
import csv
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import urllib.request

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("USA Contiguous Albers Equal Area Conic USGS")
arcpy.env.parallelProcessingFactor = "100%"

In [ ]:
data_path = "your_path"

##Change this to working directory
os.chdir(data_path)
base_url = 'https://hazards.fema.gov/nfhlv2/output/State/'

In [ ]:
# Dictionary mapping Census state FIPS codes to state abbreviations
fips_to_abbreviation = {
    '01': 'AL',  # Alabama
    '02': 'AK',  # Alaska
    '04': 'AZ',  # Arizona: No state layer available
    '05': 'AR',  # Arkansas
    '06': 'CA',  # California
    '08': 'CO',  # Colorado: No state layer available
    '09': 'CT',  # Connecticut
    '10': 'DE',  # Delaware
    '11': 'DC',  # District of Columbia
    '12': 'FL',  # Florida: No state layer available
    '13': 'GA',  # Georgia
    '15': 'HI',  # Hawaii
    '16': 'ID',  # Idaho
    '17': 'IL',  # Illinois: No state layer available
    '18': 'IN',  # Indiana
    '19': 'IA',  # Iowa
    '20': 'KS',  # Kansas
    '21': 'KY',  # Kentucky
    '22': 'LA',  # Louisiana
    '23': 'ME',  # Maine
    '24': 'MD',  # Maryland
    '25': 'MA',  # Massachusetts
    '26': 'MI',  # Michigan
    '27': 'MN',  # Minnesota
    '28': 'MS',  # Mississippi: No state layer available
    '29': 'MO',  # Missouri
    '30': 'MT',  # Montana
    '31': 'NE',  # Nebraska
    '32': 'NV',  # Nevada
    '33': 'NH',  # New Hampshire
    '34': 'NJ',  # New Jersey
    '35': 'NM',  # New Mexico: No state layer available
    '36': 'NY',  # New York
    '37': 'NC',  # North Carolina
    '38': 'ND',  # North Dakota
    '39': 'OH',  # Ohio
    '40': 'OK',  # Oklahoma
    '41': 'OR',  # Oregon
    '42': 'PA',  # Pennsylvania
    '44': 'RI',  # Rhode Island
    '45': 'SC',  # South Carolina
    '46': 'SD',  # South Dakota
    '47': 'TN',  # Tennessee
    '48': 'TX',  # Texas: No state layer available
    '49': 'UT',  # Utah
    '50': 'VT',  # Vermont
    '51': 'VA',  # Virginia
    '53': 'WA',  # Washington
    '54': 'WV',  # West Virginia
    '55': 'WI',  # Wisconsin
    '56': 'WY'   # Wyoming
}

stfips = list(fips_to_abbreviation.keys())

In [ ]:
### Unzip the nfhl
zipped_folder = "nfhl_zipped"
unzipped_folder = "nfhl_unzipped"
os.makedirs(unzipped_folder, exist_ok=True)

file_prefix = "S_POL_AR"

# Loop through each ZIP file in the folder
for zip_filename in os.listdir(zipped_folder):
    if zip_filename.endswith(".zip"):
        zip_path = os.path.join(zipped_folder, zip_filename)
        
        # Create a new folder to extract to (named after the zip file without the extension)
        extract_folder = os.path.join(unzipped_folder, os.path.splitext(zip_filename)[0])
        os.makedirs(extract_folder, exist_ok=True)
        
        try:
            # Open the ZIP file
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # Get all files in the ZIP archive
                files_in_zip = zip_ref.namelist()
                
                # Loop through the files and extract only those with the desired prefix
                for file_name in files_in_zip:
                    if file_name.startswith(file_prefix):  # Check if the file starts with the prefix
                        # Extract the file to the designated folder
                        zip_ref.extract(file_name, extract_folder)
                        print(f"Extracted: {file_name} from {zip_filename}")
            
            print(f"Successfully extracted files from: {zip_filename}")
        except Exception as e:
            print(f"Failed to extract {zip_filename}: {e}")


In [ ]:
# Set the output geodatabase path
input_folder = os.path.join(data_path, "nfhl_unzipped")
output_nfhl_gdb = os.path.join(data_path, "nfhl_footprints.gdb")

In [ ]:
# Check if the geodatabases exists
if not arcpy.Exists(output_nfhl_gdb):
    # If it doesn't exist, create the geodatabase
    arcpy.CreateFileGDB_management(os.path.dirname(output_nfhl_gdb), os.path.basename(output_nfhl_gdb))
    print(f"Geodatabase created: {output_nfhl_gdb}")
else:
    print(f"Geodatabase already exists: {output_nfhl_gdb}")

In [ ]:
folder_list = os.listdir(input_folder)

In [ ]:
all_layers = []

for folder_name in folder_list:
    shp_path = os.path.join(data_path, "nfhl_unzipped", folder_name, "S_POL_AR.shp")
    if os.path.exists(shp_path):
        all_layers.append(shp_path)
        # print(f"Added {shp_path} to state list")
    else:
        print(f"File does not exists at: {shp_path}")

In [ ]:
print(f"Merging {len(all_layers)} layers of NFHL footprint")
arcpy.management.Merge(all_layers, os.path.join(output_nfhl_gdb, "nfhl_footrpint"))

In [ ]:
# Define input and output layers
input_layer = os.path.join(output_nfhl_gdb, "nfhl_footrpint")  # Change to your actual input layer
output_layer = os.path.join(output_nfhl_gdb, "nfhl_footprint_simple")  # Change to desired output location

# Copy the input layer to preserve the original
print("Copying input layer to a new output layer")
arcpy.management.CopyFeatures(input_layer, output_layer)

# Get all fields in the copied layer
fields_to_keep = ["OBJECTID", "Shape", "Shape_Length", "Shape_Area"]  # Mandatory fields
all_fields = [f.name for f in arcpy.ListFields(output_layer) if f.name not in fields_to_keep]

# Delete unnecessary fields
if all_fields:
    print(f"Deleting unnecessary fields: {all_fields}")
    arcpy.management.DeleteField(output_layer, all_fields)

# Add the "fema_model" field
print("Adding 'fema_model' field")
arcpy.management.AddField(output_layer, "fema_model", "SHORT")

# Set all values in "fema_model" to 1
print("Updating 'fema_model' field values")
with arcpy.da.UpdateCursor(output_layer, ["fema_model"]) as cursor:
    for row in cursor:
        row[0] = 1
        cursor.updateRow(row)

print("Process completed successfully.")
